# SPAH Postcode Mapping - Final merge & export

In [1]:
import pandas as pd
import folium

### Import data

In [2]:
# define file pathways
cleaned_folder_path = r'C:\Users\scotth07\OneDrive - NHS Scotland\Documents\Networks & Projects\SPAH\Postcode Mapping\cleaned_data'
raw_folder_path = r'C:\Users\scotth07\OneDrive - NHS Scotland\Documents\Networks & Projects\SPAH\Postcode Mapping\raw_data'

patient_data_pathway = cleaned_folder_path + r'\patient_data.xlsx'
nearest_hospitals_pathway = cleaned_folder_path + r'\nearest_hospitals.xlsx' 

In [3]:
# import data
patient_df = pd.read_excel(patient_data_pathway, sheet_name='Sheet1')
nearest_hospitals_df = pd.read_excel(nearest_hospitals_pathway, sheet_name='Sheet1') 

### Merge using postcodes and export list of patients

In [4]:
# merge the two dataframes to combine patient data and nearest hospital information
merged_df = pd.merge(patient_df, nearest_hospitals_df,
                    on='patient_postcode')                     

In [5]:
merged_df.to_excel(cleaned_folder_path + r'\patient_hospitals_merged.xlsx',
                  index=False)